In [ ]:
# importing the module
import json
import re
import pandas as pd
import numpy as np
from IPython.display import display
from utils import *
from file_reader import *

In [ ]:
def get_all_project_id(file_name):
    df = pd.read_csv(file_name, index_col=0)
    #display(df.index)
    return df.index
    

project_id_list = get_all_project_id('project_df_lib.csv')


In [ ]:

def get_lib_list(path,id_list,output_df):
    lib_list=[]
    df_header=['userID','itemID','rating']
    df = pd.DataFrame(columns=df_header)
    for i in id_list:
        libs = read_file(path,i)
        for j in libs:
            df2 = pd.DataFrame({'userID': [i],
                            'itemID': [j],
                            'rating': [1]})
            df = df.append(df2, ignore_index = True)

        lib_list = lib_list+libs
    #df.to_csv(output_df, index = False)
    #display(df)    
    lib_set = set(lib_list)
    return lib_set,df

def create_lib_df(path,id_list, output_df):
    libs,temp_df = get_lib_list(path,id_list,output_df)
    display(len(temp_df))
    list_of_zeroes = [0]*(len(libs)*len(id_list))
    index = pd.MultiIndex.from_product([id_list, libs], names = ["userID", "itemID"])
    df = pd.DataFrame(index = index)
    df['rating'] = list_of_zeroes
    for i in range(0, len(temp_df)):
        #print (temp_df.loc[i,'userID'])
        df.loc[temp_df.loc[i,'userID'],temp_df.loc[i,'itemID']]=1
    df.to_csv('surprise_libraries_df.csv')
    display(df)

create_lib_df('C:\\progetti\\low-code-ard-proj\\data-scraper\\code-snippets\\', project_id_list,output_df='temp_lib.csv')
#get_lib_list('C:\\progetti\\low-code-ard-proj\\data-scraper\\code-snippets\\',project_id_list)

In [ ]:
from matplotlib.pyplot import get
from surprise import Dataset,accuracy, NormalPredictor, Reader,SVD, KNNWithMeans,KNNBasic, KNNWithZScore
from surprise.model_selection import cross_validate, train_test_split,KFold
import heapq
from collections import defaultdict
from operator import itemgetter


def precision_recall_at_k(predictions, k=10, threshold=0.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for  uid,_, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    #print(user_est_true)
    #display(user_est_true)

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        #print('relevant: ',n_rel)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        #print('recommended: ',n_rec_k)

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls






df = pd.read_csv('temp_lib.csv')
 
#display(df)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 1))
#print(df.columns)
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[df.columns], reader)
algo = KNNWithMeans(sim_options={
        'name': 'msd',
        'user_based': True,
        })
        #'min_support':5,
        #'shrinkage':0

# Retrieve the trainset.
#trainset, testset = train_test_split(data, test_size=0.25)


# define a cross-validation iterator
kf = KFold(n_splits=2)


for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0.1)
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))
    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)


# Train the algorithm on the trainset, and predict ratings for the testset
#algo.fit(trainset)
#predictions = algo.test(testset)
# We can now use this dataset as we please, e.g. calling cross_validate
#cross_validate(NormalPredictor(), data, cv=10)
#cross_validate(algo, data, measures=["RMSE", "MAE"], cv=10, verbose=True)



In [ ]:
display (predictions)

In [ ]:
def print_predictions(real, lower_bound):
    count = 0
    for i in range(0,len(predictions)):
        if(predictions[i][2]==real and predictions[i][3]>lower_bound):
            count= count +1
            print(count, predictions[i][2],predictions[i][3])
       
print_predictions(1, 0.1)

In [ ]:
import random

from surprise import accuracy, Dataset, SVD
from surprise.model_selection import GridSearchCV


df = pd.read_csv('surprise_libraries_df.csv')
 
reader = Reader(rating_scale=(0, 1))

data = Dataset.load_from_df(df[df.columns], reader)

raw_ratings = data.raw_ratings

# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(0.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print("Grid Search...")
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005]}
grid_search = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=3)
grid_search.fit(data)

algo = grid_search.best_estimator["rmse"]

# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print("Biased accuracy on A,", end="   ")
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print("Unbiased accuracy on B,", end=" ")
accuracy.rmse(predictions)